In [1]:
# dependencies go here
import pandas as pd
from sqlalchemy import create_engine
# from config import pw
import os

ModuleNotFoundError: No module named 'config'

## set up database

We set up the `minneapolis_housing` database and related tables in Postgresql:

* `neighborhood`
* `home_value`
* `crime`

Code for setting up the tables can be found in the queries.sql file accompanying this notebook.

**Note: Refactor this to work with SQLAlchemy, if time permits**

## load data
### Neighborhood datasets

In [ ]:
# Stacy start

In [ ]:
# Load Minneapolis city dataset for neighborhoods
n_path = os.path.join('.', 'data', 'Minneapolis_Neighborhoods.csv')
neighborhood = pd.read_csv(n_path)
neighborhood_df = neighborhood[['FID', 'BDNAME']].copy()
neighborhood_df.rename(columns={
    'FID': 'id',
    'BDNAME': 'neighborhood'
}, inplace=True)

'''
NOTES:
Noticed there wasn't a codebook that explained what the cryptic column names mean, so used FID based on the fact
that it is a unique ID.
'''
neighborhood_df.head()

In [ ]:
# Load Minneapolis neighborhood census dataset 
census_path = os.path.join('.', 'data', 'census_2010.xls')
census = pd.read_excel(census_path, header=None)
census_df = census.iloc[6:, :2]
census_df.rename(columns={
    0: 'neighborhood',
    1: 'population_2010'
}, inplace=True)
census_df.head()

In [ ]:
# merge the dfs together into one neighborhood df
neighborhoods = neighborhood_df.merge(census_df, how='outer', on='neighborhood')
neighborhoods.head()

In [ ]:
# Check dataframe for null values

mask = pd.isnull(neighborhoods.id)
print(neighborhoods[mask])

mask = pd.isnull(neighborhoods.population_2010)
print(neighborhoods[mask])

# No population data for South Uptown (44) or Kenwood (74)
# No ids for CARAG (87) or Kenwood (88)

In [ ]:
# deal with null values 

# Kenwood (74) should be updated with Kenwood (88)'s pop data 
neighborhoods.loc[73, 'population_2010'] = neighborhoods.loc[88, 'population_2010']
neighborhoods.tail(20)

# create id for CARAG (87)
neighborhoods.loc[87, 'id'] = '88'

# Drop rows 88, 89
neighborhoods.drop([88,89], inplace=True)

In [ ]:
# Stacy end

In [ ]:
# Jenna start
#Load Assessors Parcel Data 2019 CSV & Create dataframe
csv_file = "data/Assessors_Parcel_Data_2019.csv"
assessors_df = pd.read_csv(csv_file)
assessors_df.head()

In [ ]:
#Create a new dataframe with select columns
assessors_parcel_df = assessors_df[['NEIGHBORHOOD', 
                                    'FORMATTED_ADDRESS',
                                   'PROPERTY_TYPE',
                                   'LANDUSE',
                                   'TOTALVALUE',
                                   'BELOWGROUNDAREA',
                                   'ABOVEGROUNDAREA',
                                   'BEDROOMS',
                                   'PARCEL_AREA_SQFT',
                                   'X',
                                   'Y']]
assessors_parcel_df.head()

In [ ]:
#Rename columns

In [ ]:
#Filter out property types that are not residential

In [ ]:
#Calculated Columns

In [ ]:
# Jenna end

In [2]:
# Katrina start
#Loading data for police incidents and creating a dataframe for the data
csv_file = "data/Police_Incidents_2019.csv"
police_incidents_df = pd.read_csv(csv_file)
police_incidents_df.head()

,X,Y,publicaddress,caseNumber,precinct,reportedDate,reportedTime,beginDate,reportedDateTime,beginTime,...,enteredDate,centergbsid,centerLong,centerLat,centerX,centerY,neighborhood,lastchanged,LastUpdateDateETL,OBJECTID
0,-93.277816,44.961796,0020XX NICOLLET AVE,MP2019703050,5,2019-06-06T00:00:00.000Z,1253,2019-06-06T00:00:00.000Z,2019-06-06T12:53:00.000Z,1100,...,2019-06-07T00:00:00.000Z,18036.0,-93.277819,44.961801,-10383639,5615509,Whittier,2019-06-07T00:00:00.000Z,2019-07-25T14:36:02.000Z,7001
1,-93.230744,45.008673,0019XX CLEVELAND ST NE,MP2019703051,2,2019-06-06T00:00:00.000Z,1321,2019-06-06T00:00:00.000Z,2019-06-06T13:21:00.000Z,100,...,2019-06-07T00:00:00.000Z,17223.0,-93.230748,45.008675,-10378399,5622887,Windom Park,2019-06-07T00:00:00.000Z,2019-07-25T14:36:02.000Z,7002
2,-93.258925,44.962711,0010XX FRANKLIN AVE E,MP2019703054,3,2019-06-06T00:00:00.000Z,1423,2019-06-05T00:00:00.000Z,2019-06-06T14:23:00.000Z,1200,...,2019-06-07T00:00:00.000Z,16490.0,-93.258926,44.962713,-10381536,5615653,Ventura Village,2019-06-07T00:00:00.000Z,2019-07-25T14:36:02.000Z,7003
3,-93.273549,44.986948,NaN,MP2019703057,1,2019-06-06T00:00:00.000Z,1456,2019-06-02T00:00:00.000Z,2019-06-06T14:56:00.000Z,1745,...,2019-06-07T00:00:00.000Z,NaN,-93.273560,44.986950,-10383164,5619467,North Loop,2019-06-07T00:00:00.000Z,2019-07-25T14:36:02.000Z,7004
4,-93.290330,44.971990,0008XX DUNWOODY BLVD,MP2019703058,5,2019-06-06T00:00:00.000Z,1520,2019-06-06T00:00:00.000Z,2019-06-06T15:20:00.000Z,950,...,2019-06-07T00:00:00.000Z,16056.0,-93.290335,44.971991,-10385032,5617113,Lowry Hill,2019-06-07T00:00:00.000Z,2019-07-25T14:36:02.000Z,7005


In [6]:
# Katrina end
#Create new dataframe with certain columns
incidents_df = police_incidents_df[['description',
                                    'neighborhood',
                                    'X',
                                    'Y']]
incidents_df

,description,neighborhood,X,Y
0,OTHER THEFT,Whittier,-93.277816,44.961796
1,THEFT FROM MOTR VEHC,Windom Park,-93.230744,45.008673
2,THEFT FROM MOTR VEHC,Ventura Village,-93.258925,44.962711
3,BIKE THEFT,North Loop,-93.273549,44.986948
4,THEFT FROM MOTR VEHC,Lowry Hill,-93.290330,44.971990
...,...,...,...,...
26469,THEFT-MOTR VEH PARTS,Powderhorn Park,-93.257487,44.938648
26470,OTHER THEFT,Ventura Village,-93.262563,44.960010
26471,OTHER THEFT,CARAG,-93.290006,44.948363
26472,AUTOMOBILE THEFT,Minnehaha,-93.220297,44.909899


## transforming data

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start

In [ ]:
# Katrina end

## adding to postgres

In [ ]:
# Stacy start

In [ ]:
# Stacy end